In [1]:
import warnings

warnings.simplefilter("ignore")

import torch

from som_based_rag import SOM_Based_RAG_Util

from wiki_datasource import WikiEventsDataSource

from openai_vector_encoder import OpenAIEmbeddingsVectorEncoder

from openai_qa_chatbot import OpenAIQuestionAnswerChatBot

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [3]:
openai_key = "YOUR KEY GOES HERE"

openai_vector_encoder_id = "text-embedding-ada-002"

openai_encoded_vector_dimensions = 1536

openai_tokenizer_name = "cl100k_base" 

openai_model_name = "gpt-3.5-turbo-instruct"

vector_encoder = OpenAIEmbeddingsVectorEncoder( openai_encoded_vector_dimensions, openai_vector_encoder_id, openai_key )

In [4]:
event_years_to_fetch = [ 2022, 2023, 2024 ]
data_source = WikiEventsDataSource( event_years_to_fetch  )

In [5]:
som_driven_rag_util = SOM_Based_RAG_Util( 
                                            vector_encoder = vector_encoder,
                                            som_lattice_height = 20,
                                            som_lattice_width = 30,
                                            learning_rate = 0.3,
                                            topk_bmu_for_indexing = 10, #about 3% of the lattice grid size => 20 x 30 = 600
                                            device = device
                                        )

In [6]:
som_driven_rag_util.load_n_vectorize_data( data_source )

Vectorized Data Batch:   0%|          | 0/5 [00:00<?, ?it/s]

In [7]:
som_driven_rag_util.vectors.shape

torch.Size([458, 1536])

In [8]:
som_driven_rag_util.train_n_index_data_vectors( train_epochs = 100 )

Kohonen's SOM Train Epochs:   0%|          | 0/100 [00:00<?, ?it/s]

SOM-Based Indexed Vectors:   0%|          | 0/458 [00:00<?, ?it/s]

In [9]:
som_driven_rag_util.find_semantically_similar_data( "Who aquired Twitter in 2022?", sim_threshold = 0.8 )

[{'text': 'October 28, 2022 – Elon Musk completes his $44 billion acquisition of Twitter.',
  'sim_score': 0.8670649528503418},
 {'text': 'April 25, 2022 – Elon Musk reaches an agreement to acquire the social media network Twitter (which he later rebrands as X) for $44 billion USD, which later closes in October.',
  'sim_score': 0.8498020768165588}]

In [10]:
openai_chatbot = OpenAIQuestionAnswerChatBot( 
                                                vector_db_util = som_driven_rag_util,
                                                openai_tokenizer_name = openai_tokenizer_name,
                                                openai_model_name = openai_model_name,
                                                openai_key = openai_key,
                                                question_input_max_token_count = 100,
                                                context_trim_percent = 0.1,
                                                device = device
                                            )

In [11]:
sample_questions = [
                        "Who won the 2022 soccer world cup?",
                        "When did Sweden join NATO?",
                        "Who joined NATO in 2023?",
                        "Who joined NATO in 2024?",
                        "Which is the 31st member of NATO?",
                        "Which is the 32nd member of NATO?",
                        "Who won the Cricket World Cup in 2023?",
                        "Who defeated India in Cricket World Cup final in 2023?",
                        "Name the former prime minister of Japan that was assassinated in 2022?",
                        "When did Chandrayaan-3 land near the south pole of the Moon?",
                        "Where did Chandrayaan-3 land on the Moon?",
                        "Who acquired Twitter in 2022?",
                        "Who owns Twitter?",
                        "Who acquired Activision Blizzard in 2023?"
                   ]

In [12]:
for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question}" )
    answer = openai_chatbot.find_answer_to_question_without_context( sample_question )
    print( f"Answer: {answer}" )


Question: Who won the 2022 soccer world cup?
Answer: I do not know, as the 2022 soccer world cup has not happened

Question: When did Sweden join NATO?
Answer: Sweden is not a member of NATO.

Question: Who joined NATO in 2023?
Answer: There is no information available on which countries joined NATO in 2023

Question: Who joined NATO in 2024?
Answer: It is not possible for me to answer this question as it is only 202

Question: Which is the 31st member of NATO?
Answer: I cannot answer that question as the members of NATO can change over time due to

Question: Which is the 32nd member of NATO?
Answer: The 32nd member of NATO is Montenegro.

Question: Who won the Cricket World Cup in 2023?
Answer: As the year 2023 has not yet arrived, it is not

Question: Who defeated India in Cricket World Cup final in 2023?
Answer: I do not have information on a specific match in 2023, but according

Question: Name the former prime minister of Japan that was assassinated in 2022?
Answer: There is curr

In [13]:
sim_threshold = 0.75

for sample_question in sample_questions:
    print( f"\nQuestion: {sample_question}" )
    answer = openai_chatbot.find_answer_to_question( sample_question, sim_threshold = sim_threshold)
    print( f"Answer: {answer}" )


Question: Who won the 2022 soccer world cup?
Answer: Argentina.

Question: When did Sweden join NATO?
Answer: March 7, 2024

Question: Who joined NATO in 2023?
Answer: Finland.

Question: Who joined NATO in 2024?
Answer: Sweden officially joined NATO.

Question: Which is the 31st member of NATO?
Answer: Finland

Question: Which is the 32nd member of NATO?
Answer: Sweden

Question: Who won the Cricket World Cup in 2023?
Answer: Australia.

Question: Who defeated India in Cricket World Cup final in 2023?
Answer: Australia.

Question: Name the former prime minister of Japan that was assassinated in 2022?
Answer: Shinzo Abe

Question: When did Chandrayaan-3 land near the south pole of the Moon?
Answer: August 23, 2023.

Question: Where did Chandrayaan-3 land on the Moon?
Answer: Near the south pole.

Question: Who acquired Twitter in 2022?
Answer: Elon Musk

Question: Who owns Twitter?
Answer: Elon Musk.

Question: Who acquired Activision Blizzard in 2023?
Answer: Microsoft
